<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/neural_networks/RNN/drama_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Drama Generator

In [0]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras 
import tensorflow as tf
import os
import numpy as np

### Zbiór danych

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Załadowanie własnych danych

In [0]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

### Odczytanie treści pliku

In [44]:
# Przeczytanie i zakodowanie do formatu py2
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print('Długość tekstu: {} znaków'.format(len(text)))

Długość tekstu: 1115394 znaków


In [45]:
# Rzućmy okiem na pierwsze 250 znaków w tekście
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Kodowanie

In [0]:
vocab = sorted(set(text))
# Stworzenie mapowania unikatowych znaków do wskaźników
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [47]:
# zobaczmy na przykładzie jak nasz tekst został zakodowany
print('Text:', text[:13])
print('Encoded', text_to_int(text[:13]))

Text: First Citizen
Encoded [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [48]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


### Tworzenie przykładów treningowych

In [0]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# Tworzenie przykładów treningowych
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [0]:
def split_input_target(chunk):  # na przykład: hello
  input_text = chunk[:-1]  # hell
  target_text = chunk[1:]  # ello
  return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # używamy mapowania do zastosowania powyższej funkcji do każdego wpisu

In [52]:
for x, y in dataset.take(2):
  print('\n\nEXAMPLE\n')
  print('INPUT')
  print(int_to_text(x))
  print('\nOUTPUT')
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [0]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab to liczba unikatowych znaków
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Budowa modelu

In [54]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Tworzenie funkcji straty

In [55]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, '# (batch_size, sequence_length, vocab_size)')

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [56]:
# możemy zobaczyć, że predykcja jest tablicą 64 tablic, po jednym wpisie na wsad
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.2756020e-03  1.0941733e-03  9.9033809e-05 ... -1.8608464e-03
    1.0167472e-03 -5.2739895e-04]
  [-4.9946303e-03  4.4669895e-03 -3.8288368e-03 ... -4.7846194e-03
    2.7665924e-03 -2.8478878e-03]
  [-2.4771232e-03  3.1500990e-03 -1.3275494e-02 ... -3.0980220e-03
    2.9893944e-03 -3.8632054e-03]
  ...
  [-5.5281585e-03 -1.1321435e-02  4.3290039e-03 ...  8.0144862e-03
   -9.8271901e-04  9.7659379e-03]
  [-1.8137688e-03 -1.5505778e-03  2.6749412e-03 ...  2.5844565e-03
   -4.7952235e-03  1.0138634e-02]
  [-4.4095283e-03 -2.2342233e-03  6.0934145e-03 ...  5.6611970e-03
   -2.4895081e-03  9.8194769e-03]]

 [[-9.5171115e-04 -2.2133302e-03  2.8357576e-03 ...  3.3565369e-03
    3.4633213e-05  1.3851903e-03]
  [-3.5982074e-03  3.2581401e-03  6.0820347e-04 ... -2.9555429e-04
    1.0745848e-04 -8.9187827e-04]
  [-4.5260820e-03  8.5509857e-03 -7.4342676e-03 ...  2.6712918e-03
   -2.2839077e-03 -1.4654216e-03]
  ...
  [-1.4074701e-04  3.2960174e-03 -4.4243103e-03 ...  7.0089829e

In [57]:
# sprawdźmy pojedynczą predykcję
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-1.2756020e-03  1.0941733e-03  9.9033809e-05 ... -1.8608464e-03
   1.0167472e-03 -5.2739895e-04]
 [-4.9946303e-03  4.4669895e-03 -3.8288368e-03 ... -4.7846194e-03
   2.7665924e-03 -2.8478878e-03]
 [-2.4771232e-03  3.1500990e-03 -1.3275494e-02 ... -3.0980220e-03
   2.9893944e-03 -3.8632054e-03]
 ...
 [-5.5281585e-03 -1.1321435e-02  4.3290039e-03 ...  8.0144862e-03
  -9.8271901e-04  9.7659379e-03]
 [-1.8137688e-03 -1.5505778e-03  2.6749412e-03 ...  2.5844565e-03
  -4.7952235e-03  1.0138634e-02]
 [-4.4095283e-03 -2.2342233e-03  6.0934145e-03 ...  5.6611970e-03
  -2.4895081e-03  9.8194769e-03]], shape=(100, 65), dtype=float32)


In [58]:
# spójrzmy na predykcję z pierwszego kroku
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# jest tu 65 wartości reprezentowanych przez prawdopodobieństwo wystąpienia każdego znaku jako następny

65
tf.Tensor(
[-1.2756020e-03  1.0941733e-03  9.9033809e-05 -3.7100830e-04
  5.6809927e-03 -7.4793526e-04 -9.2137590e-05 -2.6758232e-03
  1.5958657e-03 -1.8048374e-03 -3.5606974e-03  1.8626013e-03
 -4.3759678e-04  2.7684614e-03 -2.6411107e-03 -2.3570445e-03
  2.8696519e-03 -5.5329211e-04 -3.0692404e-03  3.2588774e-03
 -2.6513748e-03  1.1792282e-03  3.9605140e-03  9.3658920e-03
  1.7634440e-05  4.3875352e-03  4.2639792e-04  2.0342353e-03
 -3.1153311e-04 -2.5496816e-03 -1.0030870e-03  2.3485881e-03
  1.4894512e-03 -2.5002579e-03  3.4288352e-04  1.2853282e-03
  1.7979086e-03 -3.2415984e-03 -4.9822492e-04 -1.0975995e-03
 -4.8717051e-03 -3.2724312e-03  4.2283107e-03 -1.7875977e-03
  3.7031602e-03 -6.1697913e-03 -4.7452246e-05 -2.2232460e-03
 -9.6392490e-05 -9.8107522e-04 -3.1455180e-03 -4.2782456e-04
  2.3093324e-03 -1.1087297e-03  7.4675274e-03  2.2986417e-03
 -7.7618472e-04  1.4552096e-03  4.5798169e-03  4.5486977e-03
  6.1028539e-03 -1.5859671e-04 -1.8608464e-03  1.0167472e-03
 -5.273989

In [59]:
sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

"DRieLPnhj!JixCDKJ kDPUlxf?E-yPUhWDrOMbkgfY&VeFsa:, g'Ii&X$3CYsHCcMouTapnrDq;zJ:cW;yrIeJd!3yh&FSUcvc'"

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

### Kompilacja modelu

In [0]:
model.compile(optimizer='adam', loss=loss)

### Tworzenie checkpointów

In [0]:
# Słownik, w którym checkpointy będą zapisywane
checkpoint_dir = './training_checkpoints'
# Nadanie nazwy dla plików checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Trening

In [63]:
history = model.fit(data, epochs=10, callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/10
172/172 [==============================] - 34s 198ms/step - loss: 2.5871
Epoch 2/10
172/172 [==============================] - 32s 188ms/step - loss: 1.8930
Epoch 3/10
172/172 [==============================] - 32s 188ms/step - loss: 1.6438
Epoch 4/10
172/172 [==============================] - 32s 188ms/step - loss: 1.5093
Epoch 5/10
172/172 [==============================] - 32s 189ms/step - loss: 1.4263
Epoch 6/10
172/172 [==============================] - 32s 188ms/step - loss: 1.3688
Epoch 7/10
172/172 [==============================] - 32s 188ms/step - loss: 1.3238
Epoch 8/10
172/172 [==============================] - 32s 187ms/step - loss: 1.2850
Epoch 9/10
172/172 [==============================] - 32s 188ms/step - loss: 1.2480
Epoch 10/10
172/172 [==============================] - 32s 183ms/step - loss: 1.2123


### Załadowanie modelu

In [0]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [0]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [0]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint('./training_checkpoints/ckpt_' + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

### Generowanie tekstu

In [0]:
def generate_text(model, start_string):

  # liczba znaków do wygenerowania
  num_generate = 800

  # konwersja stringów na liczby (wektoryzacja)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # pusty string do przechowania naszych wyników
  text_generated = []

  # Niższe temperatury powodują bardziej przewidywalny tekst.
  # Wyższe temperatury powodują bardziej niespodziewany tekst.
  temperature = 1.0

  # batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # usunięcie wymiaru wsadu
    predictions = tf.squeeze(predictions, 0)

    # użycie rozkładu kategorycznego do predykcji znaku zwróconego przez model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

    # Przepuszczamy przewidziany znak jako następne wejście do modelu
    # wraz z poprzednim ukrytym stanem
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [75]:
inp = input('Wpisz łańcuch początkowy: ')
print(generate_text(model, inp))

Wpisz łańcuch początkowy: be or not to be
be or not to bed your lord.

LUCENTIO:
Farewell: I hopper else I am
I will revenge you to our life unto his urboor fortunce, and a curst harrial men.

ROMEO:
Not with me o penine, till what do for the water,
As him and whine woman! I will find me to
me not, sir.

MERCUTIO:
Hence,!
O meeping means are alrosled, I thank you your arms!
Soul should he said that is the river and resign ones,
No charges with debt. But how did you Richard stand ourselves, as the words of mine:'
For now that I might find thee to a word;
So, now for mad I give it not to hear.

KING RICHARD II:
Nay, I will die in these;
For that we shall not, hirse! some deally
place I mean, that they safell'd I will cheer;
And so pray you with marrial death.

Third Citizen:
To-morrow looks; which I think you hence to he
All then gone time:
Since 


### Źródło:
[https://www.youtube.com/watch?v=tPYj3fFJGjk](https://www.youtube.com/watch?v=tPYj3fFJGjk)